In [ ]:
pip install deepctr-torch

In [ ]:
pip install pandas 

In [ ]:
conda install -c conda-forge ipywidgets

In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.9f}'.format


In [2]:
rs_table = pd.read_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\DeepFM_DataSet_Basic_content_id_1218_1551.csv')
rs_table

,subsr,content_id,ct_cl,genre_of_ct_cl,template_A_TopGroup,template_B_TopGroup,template_C_TopGroup
0,65941000,id_0,영화,멜로,"Occult, Psychological thriller","Action comedy, Action drama, Action thriller, ...",NaN
1,65941000,id_1,영화,액션/어드벤쳐,"Action comedy, Action crime, Action drama, Act...","Adventure, Military Action, Action crime, Acti...","Military Action, Slasher"
2,65941000,id_2,영화,애니메이션,Occult,Martial Arts,Martial Arts
3,65941000,id_3,영화,드라마,"Action comedy, Romantic comedy","Action comedy, Action crime, Action drama, Act...",NaN
4,65941000,id_4,영화,애니메이션,"Folk Horror, Occult, Psychological thriller, M...","Adventure, Action crime, Action drama, Action ...",Historical romance
...,...,...,...,...,...,...,...
2773225,65198000,id_5713,TV드라마,기타,"Adventure, Video game movies, Action comedy, A...","Action comedy, Action crime, Action crime, Act...","Action crime, Legal drama"
2773226,65198000,id_5714,TV드라마,기타,"Adventure, Video game movies, Action comedy, A...","Action comedy, Action crime, Action crime, Act...","Action crime, Legal drama"
2773227,65198000,id_5715,TV드라마,기타,"Adventure, Video game movies, Action comedy, A...","Action comedy, Action crime, Action crime, Act...","Action crime, Legal drama"
2773228,65198000,id_5716,TV드라마,기타,"Adventure, Video game movies, Action comedy, A...","Action comedy, Action crime, Action crime, Act...","Action crime, Legal drama"


In [3]:
train = pd.read_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\DeepFM_DataSet_train.csv')
test = pd.read_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\DeepFM_DataSet_test.csv')

train = train[['subsr','content_id','user_preference','TimeGroup']]
test = test[['subsr','content_id','user_preference','TimeGroup']]

In [4]:
def prcs_make_model_set(rs_table: pd.DataFrame, input_table: pd.DataFrame) -> pd.DataFrame:
    merged_set = pd.merge(left = rs_table, right = input_table, on = ['subsr','content_id'],how='left')
    merged_set.replace({np.NaN:None},inplace=True)

    merged_set['user_preference'] = merged_set['user_preference'].fillna(0)
    merged_set['user_preference'] = merged_set['user_preference'].astype(int)


    return merged_set




In [5]:
train_set = prcs_make_model_set(rs_table = rs_table, input_table = train)
test_set = prcs_make_model_set(rs_table = rs_table, input_table = test)


In [6]:
train

,subsr,content_id,user_preference,TimeGroup
0,65941000,id_188,100,pm
1,66873000,id_188,100,night
2,66873000,id_188,14,night
3,61689000,id_188,100,pm
4,61619000,id_1758,95,am
...,...,...,...,...
10652,60169000,id_1158,100,night
10653,66317000,id_3470,100,night
10654,66317000,id_3470,41,night
10655,66900000,id_4062,76,pm


In [7]:

import torch
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM
import numpy as np
import ast

import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [8]:
args = {
  'threshold':30
  }

In [9]:
# 문자열 -> 리스트 로변경
def String2List_2(df: pd.DataFrame) -> pd.DataFrame :
  template_list = ['template_A_TopGroup','template_B_TopGroup','template_C_TopGroup']
  for template in template_list:
     df[template] = df[template].apply(lambda elm : elm.split(', ') if elm is not None else None)
  
  return df



In [10]:
train_set = String2List_2(train_set)
test_set = String2List_2(test_set)



In [11]:
def prcs_labeling_liked(user_preference: int) -> int:
  return 1 if user_preference >= args['threshold'] else 0


In [12]:
train_set['liked'] = train_set['user_preference'].apply(prcs_labeling_liked)
test_set['liked'] = test_set['user_preference'].apply(prcs_labeling_liked)


In [13]:
from tqdm.notebook import tqdm
tqdm.pandas()

all_genre_list = ['Adventure', 'Disaster', 'Martial Arts', 'Military Action', 'Spy and Espionage',
                  'Superhero', 'Video game movies', 'Action comedy', 'Action crime', 'Action drama',
                  'Action-horror', 'Action thriller', 'Docudrama', 'Melodrama', 'Teen drama', 'Medical drama',
                  'Legal drama', 'Religious drama', 'Sports drama', 'Political drama', 'Anthropological drama',
                  'Philosophical drama', 'Contemporary and urban fantasy', 'Epic Fantasy', 'Fairy Tale', 'Dark Fantasy',
                  'Ghost', 'Zombie', 'Werewolf', 'Vampire', 'Monster', 'Slasher', 'Splatter and Gore', 'Body Horror',
                  'Folk Horror', 'Occult', 'Found Footage', 'Outbreak', 'Historical romance', 'Regency romance', 'Romantic drama',
                  'Romantic comedy', 'Chick Flick', 'Fantasy romance', 'Space Opera or epic sci-fi', 'Utopia', 'Dystopia', 'Contemporary Sci-Fi',
                  'Cyberpunk', 'Steampunk', 'Psychological thriller', 'Mystery', 'Film noir']

def MakeModelDataSet(user: pd.DataFrame) -> pd.DataFrame:

    genre_df = pd.DataFrame(0, index=user.index, columns=all_genre_list)


    templates = user[['template_A_TopGroup', 'template_B_TopGroup', 'template_C_TopGroup']].apply(lambda row: list(set(item for sublist in row.dropna() for item in sublist)), axis=1)

    for genre in tqdm(all_genre_list, desc="Processing genres"):
        genre_df[genre] = templates.apply(lambda x: 1 if genre in x else 0)

    user.drop(columns=['template_A_TopGroup', 'template_B_TopGroup', 'template_C_TopGroup'], inplace=True)

    returned_df = pd.concat([user,genre_df],axis=1)

    return returned_df


In [ ]:
train_set = MakeModelDataSet(train_set)
test_set = MakeModelDataSet(test_set)

In [15]:
train_set.to_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\train_one_hot_enc_1218_1738.csv',index=False)
test_set.to_csv(r'E:\VOD추천 프로젝트 데이터\DATA\12.18\test_one_hot_enc_1218_1738.csv',index=False)


In [22]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM


if __name__ == "__main__":

    # 8월 사용자의 전체 데이터 학습
    data = train_set.copy()

    sparse_features = ["subsr","content_id" ,"ct_cl", "genre_of_ct_cl", "TimeGroup"] + all_genre_list

    target = ['liked']

    label_encoders = {}

    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
        label_encoders[feat] = lbe

    # label인코더 저장
    torch.save(label_encoders,'./label_encoders_1211.pth')

    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features]

    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns

    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # train_model_input = {name: data[name][:train_size] for name in feature_names}
    # test_model_input = {name: data[name][train_size:] for name in feature_names}

    train_model_input = {name: data[name] for name in feature_names}

    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

In [23]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)
model.compile("adam", "binary_crossentropy", metrics=['accuracy'])

    # Assuming 'liked' is a binary column
history = model.fit(train_model_input, data[target].values, batch_size=256, epochs=1, verbose=2, validation_split=0.2)

cpu
Train on 2222279 samples, validate on 555570 samples, 8681 steps per epoch
Epoch 1/1
276s - loss:  0.0044 - accuracy:  0.9983 - val_accuracy:  0.9999


In [ ]:
torch.save(model,'./DeepFM_epoch_1_1211.h5')


In [ ]:
# # 9월 데이터 중, 8월의 시청 기록이 있는 사용자에 한에서만 맞춤.
# user_08_list = train_set['subsr'].unique().tolist()
# test_data = test_set[test_set['subsr'].isin(user_08_list)]
# test_data.reset_index(drop=True,inplace=True)
# test_model_input = test_data.copy()
# test_model_input.drop(columns=['liked'],inplace=True)

In [24]:
for feat, lbe in label_encoders.items():
    test_set[feat] = lbe.transform(test_set[feat])

In [26]:
  
sparse_features = ["subsr", "content_id","ct_cl", "genre_of_ct_cl","TimeGroup"] + all_genre_list
    # Count #unique features for each sparse field
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features]

 # DeepFM linear_feature 설정

linear_feature_columns = fixlen_feature_columns
    # DeepFM dnn_feature 설정
dnn_feature_columns = fixlen_feature_columns


feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

test_model_input = {name: test_set[name] for name in feature_names}

In [33]:
pred_ans = model.predict(test_model_input,batch_size=256)
pred_labels = (pred_ans > 0.5).astype(int)
# accuracy = accuracy_score(test_data[target].values, pred_labels)
# print("Test Accuracy:", round(accuracy, 4))

In [ ]:
# test_set

In [34]:
result_df = test_set[['subsr', 'content_id','liked']].copy()
result_df['predicted_liked'] = pred_labels
result_df['predicted_rate'] = pred_ans
result_df.reset_index(drop=True,inplace=True)
result_df

,subsr,content_id,liked,predicted_liked,predicted_rate
0,329,0,0,0,1.862956e-10
1,329,1,0,0,1.736957e-09
2,329,1112,0,0,2.076543e-09
3,329,2223,0,0,7.471627e-10
4,329,3334,0,0,9.611985e-10
...,...,...,...,...,...
2775023,251,5239,0,0,1.916313e-09
2775024,251,5240,0,0,1.939768e-09
2775025,251,5241,0,0,1.939428e-09
2775026,251,5242,0,0,1.941848e-09


In [51]:
result_df.to_csv('./10월사용자예측값.csv',index=False)

In [56]:
result_df_cp = result_df.copy()
result_df_cp['subsr'] =  result_df_cp['subsr'].apply(lambda elm: label_encoders.get('subsr').inverse_transform([elm])[0])

In [58]:
result_df_cp['content_id'] = result_df_cp['content_id'].apply(lambda elm: label_encoders.get('content_id').inverse_transform([elm])[0])

In [65]:
result_df_cp.drop(columns=['inverse_content_id'],inplace=True)
result_df_cp

,subsr,content_id,liked,predicted_liked,predicted_rate
0,65941000,id_0,0,0,0.000000000
1,65941000,id_1,0,0,0.000000002
2,65941000,id_2,0,0,0.000000002
3,65941000,id_3,0,0,0.000000001
4,65941000,id_4,0,0,0.000000001
...,...,...,...,...,...
2775023,65198000,id_5713,0,0,0.000000002
2775024,65198000,id_5714,0,0,0.000000002
2775025,65198000,id_5715,0,0,0.000000002
2775026,65198000,id_5716,0,0,0.000000002


In [70]:
result_dict = (
    result_df_cp.groupby('subsr')
    .apply(lambda x: {x.iloc[0]['subsr']: {row['content_id']: row['predicted_rate'] for _, row in x.iterrows()}})
    .tolist()
)

In [79]:
for subdict in result_dict:
    for key2,subdict2 in subdict.items():
        result_dic ={}
        acc_dict = {}        
        # dict_a =  value_3
        # dict_b = key_table
        for key3, value3 in subdict2.items():
            for key_b, value_b in key_table.items():
                if key_b in value3:
                    result_dic[value_b] = result_dict.get(value_b,0) + value3[key_b]
                    acc_dict[key_b] = value3[key_b]


            #    if key3 in key_table:
            #         result_dic[value3] = result_dic.get(value3,0) + 


TypeError: argument of type 'float' is not iterable

In [77]:
'id_0' in key_table

True

In [64]:
key_table = pd.read_pickle(r'C:\Users\USER\Desktop\DeepFM\key_table_dic.pickle')
key_table

{'id_0': 'uq_0',
 'id_1': 'uq_1',
 'id_2': 'uq_2',
 'id_3': 'uq_3',
 'id_4': 'uq_4',
 'id_5': 'uq_5',
 'id_6': 'uq_6',
 'id_7': 'uq_7',
 'id_8': 'uq_8',
 'id_9': 'uq_9',
 'id_10': 'uq_9',
 'id_11': 'uq_10',
 'id_12': 'uq_11',
 'id_13': 'uq_12',
 'id_14': 'uq_13',
 'id_15': 'uq_14',
 'id_16': 'uq_15',
 'id_17': 'uq_16',
 'id_18': 'uq_17',
 'id_19': 'uq_18',
 'id_20': 'uq_19',
 'id_21': 'uq_20',
 'id_22': 'uq_21',
 'id_23': 'uq_22',
 'id_24': 'uq_23',
 'id_25': 'uq_24',
 'id_26': 'uq_25',
 'id_27': 'uq_26',
 'id_28': 'uq_27',
 'id_29': 'uq_28',
 'id_30': 'uq_29',
 'id_31': 'uq_30',
 'id_32': 'uq_31',
 'id_33': 'uq_32',
 'id_34': 'uq_33',
 'id_35': 'uq_34',
 'id_36': 'uq_35',
 'id_37': 'uq_36',
 'id_38': 'uq_37',
 'id_39': 'uq_38',
 'id_40': 'uq_39',
 'id_41': 'uq_40',
 'id_42': 'uq_41',
 'id_43': 'uq_42',
 'id_44': 'uq_43',
 'id_45': 'uq_44',
 'id_46': 'uq_45',
 'id_47': 'uq_46',
 'id_48': 'uq_47',
 'id_49': 'uq_48',
 'id_50': 'uq_49',
 'id_51': 'uq_50',
 'id_52': 'uq_51',
 'id_53': 'uq_

In [ ]:
# 6688500

In [46]:
# label_encoders.get('content_id').inverse_transform([251])

array(['id_1223'], dtype=object)

In [36]:
from sklearn.metrics import precision_score

precision_per_user = result_df.groupby('subsr').apply(
    lambda group: precision_score(group['liked'],group['predicted_liked'])
)
overall_precision = precision_per_user.mean()

print('10월 유저별 precision값: ', precision_per_user)
print('10월 전체 사용자에 대한 평균 Precision값: ', overall_precision)

print('갱신됨')

10월 유저별 precision값:  subsr
0      0.000000
1      0.000000
2      0.428571
3      0.000000
4      0.000000
         ...   
480    1.000000
481    0.250000
482    0.000000
483    0.000000
484    1.000000
Length: 485, dtype: float64
10월 전체 사용자에 대한 평균 Precision값:  0.2337657146202028
갱신됨
